In [ ]:
import pandas as pd
from IPython.display import Markdown
from plotly import graph_objects as go

from summer2 import AgeStratification, Overwrite
from summer2 import CompartmentalModel
from summer2.parameters import Parameter
from summer2.functions.time import get_sigmoidal_interpolation_function

from tb_incubator.constants import set_project_base_path
from tb_incubator.input import get_birth_rate, get_pop_death_data, get_death_rates, get_population_entry_rate, get_death_adjs
from tb_incubator.model import add_latency_flow, add_infection_flow

pd.options.plotting.backend = "plotly"
project_paths = set_project_base_path("../tb_incubator/")

## Population data

Estimates for the total population, deaths, and birth rate were obtained from the United Nations World Population Prospects [@unwpp2024].

In [ ]:
# Load age-stratified, population and death data
pop_death, description = get_pop_death_data()
target_pops = pop_death.groupby(level=[0]).sum()["population"]
Markdown(description)

In [ ]:
# Load death rates
death_rates, description = get_death_rates()
Markdown(description)

In [ ]:
death_rates.plot()

## Model construction
This is arbitrary, but including some epidemiological transitions in the background.

In [ ]:
# Arbitrary base model construction
model_comps = ["susceptible", "early latent", "late latent", "infectious", "recovered"]
model_times = [1850.0, 2024.0]
model = CompartmentalModel(
    times=model_times,
    compartments=model_comps,
    infectious_compartments=["infectious"],
)

# Note that we need some population at the start to avoid dividing by zero in the force of infection calculation
model.set_initial_population({"susceptible": 1.0})

# TB natural history
model.add_death_flow("TB death", Parameter("death rate"), "infectious")
model.add_transition_flow("self_recovery", Parameter("self recovery rate"), "infectious", "recovered")

# Demographic transitions
model.add_universal_death_flows("population_death", Parameter("universal death"))  # Placeholder to overwrite later
model.add_replacement_birth_flow("replacement_birth", "susceptible")

# Infection
add_infection_flow(model)

# Latency progression
add_latency_flow(model)

In [ ]:
# Age stratification
agegroup_request = [[0, 4], [5, 14], [15, 34], [35, 49], [50, 100]]
age_strata = [i[0] for i in agegroup_request]
age_strat = AgeStratification("age", age_strata, model_comps)
death_adjs = get_death_adjs(death_rates, age_strata=age_strata)
age_strat.set_flow_adjustments("population_death", death_adjs)

# Apply age stratification with age-specific death rate functions of time
model.stratify_with(age_strat)

In [ ]:
# Calculate population entry rates 
entry_rate, description = get_population_entry_rate(pop_death, 1850)

# Add births as additional entry rate (split imports in case the susceptible compartments are further stratified later)
model.add_importation_flow("births", entry_rate, dest="susceptible", split_imports=True, dest_strata={"age": "0"})
Markdown(description)

In [ ]:
params = {
    "death rate": 0.2,
    "universal death": 1.0,
    "stabilisation rate": 1.0,
    "early activation rate": 1.0,
    "late activation rate": 1.0,
    "contact rate": 0.0001,
    "self recovery rate": 0.2,
    "rr_infection_latent": 0.2,
    "rr_infection_recovered": 0.2,
}

In [ ]:
# Track populations
age_pop_outputs = [model.request_output_for_compartments(s, model_comps, strata={"age": str(s)}) for s in age_strata]

## Demographic results

In [ ]:
# Run and inspect results
model.run(params)
fig = model.get_derived_outputs_df().plot.area()
fig.add_trace(go.Scatter(x=target_pops.index, y=target_pops, name="target", mode="markers", marker=dict(color="black", size=2.0)))